 Take some parts from Kasper's example

In [1]:
from pyproj import Proj, transform
from re import sub

inProj = Proj(init='EPSG:25832')
outProj = Proj(init='epsg:4326')

def transformer(x,y):
    # point is on the form "722942.66 6173097.7"
    ll = transform(inProj, outProj, x,y)
    return str(ll[0])+' '+str(ll[1])

def geom_transformer(geom):
    reg_ex="([0-9.]+) ([0-9.]+)" # pick out two numbers with a space inbetween
    return sub(reg_ex, lambda m: transformer(m.group(2), m.group(1)), geom)

geom_transformer("POINT (722942.66 6173097.7)")

'POINT (54.48945670835061 4.579419078407395)'

In [2]:
# import data as: wfs-kbhkort.kk.dk/k101/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=k101:erhv_gravetilladelser_aktiv&outputFormat=csv
import os
import csv

our_headers = {'sagsnr':'INT', 
               'grave_art':'VARCHAR(100)', 
               'bygherre':'VARCHAR(100)', 
               'wkb_geometry':'GEOMETRY NOT NULL SRID 4326'}

def makeSQLFile(file_name, headers=our_headers):
    cwd = os.getcwd() # current working directory
    inputfile = open(f"{cwd}/{file_name}.csv","r")
    outputfile = open(f"{cwd}/{file_name}.sql", "w+")
    table = createTable(file_name, headers)
    inserts = makeInsertStatements(file_name, headers, inputfile, outputfile)
    outputfile.write(table)
    outputfile.write(inserts)
    outputfile.close()
    
our_headers

{'sagsnr': 'INT',
 'grave_art': 'VARCHAR(100)',
 'bygherre': 'VARCHAR(100)',
 'wkb_geometry': 'GEOMETRY NOT NULL SRID 4326'}

In [3]:
def createTable(name, headers):
    sql = f"drop table if exists {name};\n"
    sql += f"create table {name} (\n"
    sql += ", \n".join([f"\t{header} {sql_type}" for header,sql_type in headers.items()])
    sql += f",\n\tprimary key({list(headers.keys())[0]})\n);\n"
    print(f'created table {name}')
    return sql
print(createTable('digging', our_headers))

created table digging
drop table if exists digging;
create table digging (
	sagsnr INT, 
	grave_art VARCHAR(100), 
	bygherre VARCHAR(100), 
	wkb_geometry GEOMETRY NOT NULL SRID 4326,
	primary key(sagsnr)
);



In [4]:
def valueOf(v, sql_type):
    if sql_type == "INT":
        return v
    if sql_type[0:8] == "GEOMETRY":
        c = geom_transformer(v) # This is where I convert from EPSG:25832' to 'epsg:4326'
        return f'ST_GeomFromText("{c}", 4326)'
    return f'"{v}"'
print( valueOf(77,'INT'))
print( valueOf("POINT (722942.66 6173097.7)", 'GEOMETRY NOT NULL SRID 4326') )
print( valueOf('Fibernet', 'VARCHAR(100)'))

77
ST_GeomFromText("POINT (54.48945670835061 4.579419078407395)", 4326)
"Fibernet"


Making SQL files

In [5]:
def makeInsertStatements(name, headers, infile, outfile):
    print("Writing inserts ",end='')
    headerline = infile.readline()
    csv_headers = headerline.rstrip().split(",")
    headerIndex = { h : csv_headers.index(h) for h in headers.keys()}
    sql = ""
    csv_in = csv.reader(infile, delimiter=',', quotechar='"')
    for row in csv_in:
        print('.',end='')
        if row[-1] != "": # some positions are missing
            sql_values = [valueOf(row[headerIndex[k]], headers[k]) for k in headers.keys()]
            values_combined = ','.join(sql_values)
            columns = ','.join(headers.keys())
            sql += f"INSERT INTO {name} ( {columns} ) VALUES ({values_combined});\n"
    print("Done!")
    return sql

In [6]:
the_headers = {'id':'INT', 
               'byomraade':'VARCHAR(100)', 
               'delomraade':'VARCHAR(100)', 
               'm2':'INT', 
               'wkb_geometry':'GEOMETRY NOT NULL SRID 4326'}
makeSQLFile('f_udsatte_byomraader', headers = the_headers)

In [ ]:
the_headers = {'FID':'VARCHAR(100)', 
               'areal_id':'INT', 
               'park_id':'INT', 
               'parktype':'VARCHAR(100)', 
               'navn_parker':'VARCHAR(100)', 
               'undernavn':'VARCHAR(100)', 
               'bydelsnavn':'VARCHAR(100)', 
               'ejerforhold':'VARCHAR(100)', 
               'ejer':'VARCHAR(100)', 
               'areal':'INT', 
               'wkb_geometry':'GEOMETRY NOT NULL SRID 4326'}

makeSQLFile('parkregister', headers = the_headers)

created table parkregister
Writing inserts ........................................................................................................................................................................

In [7]:
the_headers = {'FID':'VARCHAR(100)', 
               'id':'INT', 
               'traeart':'VARCHAR(100)', 
               'slaegt':'VARCHAR(100)', 
               'dansk_navn':'VARCHAR(100)', 
               'slaegtsnavn':'VARCHAR(100)', 
               'ejer':'VARCHAR(100)', 
               'bydelsnavn':'VARCHAR(100)', 
               'vejstatus':'INT', 
               'wkb_geometry':'GEOMETRY NOT NULL SRID 4326'}

makeSQLFile('gadetraer', headers = the_headers)

created table gadetraer
Writing inserts ..................................................Done!


In [8]:
the_headers = {'FID':'VARCHAR(100)', 
               'id':'INT', 
               'vej':'VARCHAR(100)', 
               'vejid':'VARCHAR(100)', 
               'fra':'INT', 
               'til':'INT', 
               'broklasse':'INT', 
               'frihoejde':'VARCHAR(100)', 
               'wkb_geometry':'GEOMETRY NOT NULL SRID 4326'}

makeSQLFile('tungvognsnet', headers = the_headers)

created table tungvognsnet
Writing inserts ..................................................Done!


In [9]:
the_headers = {'FID':'VARCHAR(100)', 
               'id':'INT', 
               'vejkode':'INT', 
               'vejnavn':'VARCHAR(100)', 
               'park_id':'INT',
               'bydel':'VARCHAR(100)', 
               'stativ_type':'VARCHAR(100)', 
               'stativ_placering':'VARCHAR(100)', 
               'stativ_udformning':'VARCHAR(100)', 
               'cykler_retning':'VARCHAR(100)', 
               'antal_pladser':'INT',
               'stativ_tilstand':'VARCHAR(100)', 
               'wkb_geometry':'GEOMETRY NOT NULL SRID 4326'}

makeSQLFile('cykelstativ', headers = the_headers)

created table cykelstativ
Writing inserts ..................................................Done!


In [10]:
the_headers = {'FID':'VARCHAR(100)', 
               'id':'INT',  
               'vejid':'VARCHAR(100)', 
               'vejkode':'INT', 
               'vejnavn':'VARCHAR(100)', 
               'wkb_geometry':'GEOMETRY NOT NULL SRID 4326'}

makeSQLFile('vejflade', headers = the_headers)

created table vejflade
Writing inserts ..................................................Done!
